$$
\kappa(u) = \left(\frac{3u(1-u)}{3u^3 + (1-u)^3}\right)^2  + \mu
$$
$$
\begin{equation}
\left\{
	\begin{aligned}
	&-\nabla\cdot(\kappa(u)\nabla u) = f, &x \in \Omega, \\
	&u = 0, &x \in \partial\Omega
\end{aligned}
\right.
\end{equation}
$$

$$
	f(x; i^\star) = 100 \exp(-50 ||x - x_{c, i^\star}||_2^2)\mathbf{1}_{\Omega_{i^\star}}(x)
$$

In [1]:
import pandas as pd
from utils import fd_solve_nlinear, force
import numpy as np
from tqdm import tqdm

GridSize = 64
area = ((0, 0), (1, 1))
forces, mus, refs = [], [], []
xx, yy = np.meshgrid(
    np.linspace(area[0][0], area[1][0], GridSize),
    np.linspace(area[0][1], area[1][1], GridSize)
)
N = 100

with tqdm(total=N, desc='Generating:') as pbar:
	while len(refs) < N:
		center = np.random.uniform(0.1, 0.9, 2)
		mu = np.random.uniform(0.1, 1)
		f = force(xx, yy, center)
		sol = fd_solve_nlinear(GridSize, area, mu, center)

		if not sol is None:
			refs.append(sol.reshape(GridSize, GridSize))
			forces.append(f)
			mus.append(mu * np.ones((GridSize, GridSize)))
			pbar.update(1)

forces = np.stack(forces)
mus = np.stack(mus)
refs = np.stack(refs)



Generating:: 100%|██████████| 100/100 [00:45<00:00,  2.21it/s]


In [2]:
from pathlib import Path

p =Path(f'./fit_nlinear/DLData/{GridSize}')
if not p.is_dir():
	p.mkdir(parents=True)
np.save(f'./fit_nlinear/DLData/{GridSize}/F.npy', forces)
np.save(f'./fit_nlinear/DLData/{GridSize}/MU.npy', mus)
np.save(f'./fit_nlinear/DLData/{GridSize}/U.npy', refs)


In [ ]:
from FVM.src.UniformICD import UniformFVM
from FVM.src.Problem.test_uniform import *
from FVM.src.utils import show_errors

# Test zxp case !!!
case_ns = list(range(10))
print(f"|{'Method': ^8s}|{'Case ID': ^8s}|{'Mesh Type': ^18s}|{'hmesh': ^11s}|{'error_max': ^11s}|{'ratio': ^7s}|{'error_l2': ^11s}|{'ratio': ^7s}|{'error_h1': ^11s}|{'ratio': ^7s}|")
print(f'|{":--:": ^8s}|{":--:": ^8s}|{":--:": ^18s}|{":--:": ^11s}|{":--:": ^11s}|{":--:": ^7s}|{":--:": ^11s}|{":--:": ^7s}|{":--:": ^11s}|{":--:": ^7s}|')
for case_n in case_ns:
	mesh_type = 'quadrangle_uniform'
		# triangle_classic triangle_random triangle_uniform triangle_kershaw
		# quadrangle_uniform quadrangle_random  quadrangle_kershaw
	n_levels = 5
	hmeshes = []
	errors_max = []
	errors_l2 = []
	errors_h1 = []

	for i in range(n_levels):
		N = 2**(i+3)
		problem = zxpProblem(N, N, ((0, 0), (1, 1)), case_n, 1e-9)
		solver = UniformFVM(((0, 0), (1, 1)), N, N, problem)
		solver.solve(solver_name=None)
		hmesh, error_max, error_l2, error_h1 = solver.compute_errors()

		hmeshes.append(hmesh)
		errors_max.append(error_max)
		errors_l2.append(error_l2)
		errors_h1.append(error_h1)

	for s in show_errors(hmeshes, errors_max, errors_l2, errors_h1):
		print(f'|{"ICD": ^8s}|{case_n: ^8}|{mesh_type: ^18s}|' + s)

In [ ]:
from Problems.NonlinearProblem import *
# from plots import my_plot
from UniformICD import UniformFVM

def fvm_solve_nlinear(GridSize, area, mu, center=(0.5, 0.5), 
					Picard_maxiter=1000, eps=1e-7):
	u0 = np.random.rand(GridSize, GridSize)
	problem = NonLinearProblem(None, center, N, area)
	solver = UniformFVM(area, N, N, problem)
	solver.solve(solver_name = None)
	u0,  b = solver.ua, solver.b

	for i in range(Picard_maxiter):
		problem = NonLinearProblem(kappa(u0.reshape(GridSize, GridSize), mu), center, N, area)

		solver = UniformFVM(area, N, N, problem)
		solver.solve(solver_name = None)
		newA = solver.A.tocsr()
		newu = solver.ua

		# delta = ((newu - u0)**2 * h**2).sum()
		delta = np.linalg.norm(newu - u0)
		# error = ((newA @ u0 - b)**2 * h**2).sum()
		error = np.linalg.norm(newA @ u0 - b)
		print(f"Itr: {i}\t Delta: {delta:.3e}\t Error: {error:.3e}\t")

		if delta <= eps or error <= eps:
			u = newu.reshape(GridSize, GridSize)
			break
		else:
			u0, A0 = newu, newA
	return u


N = 128
mu = 0.1
center = (0.1, 0.1)
area = ((0, 0), (1, 1))

solution = fvm_solve_nlinear(N, area, mu)

In [ ]:
# import segmentation_models_pytorch as smp
from SegModel import CNN
import torch
GridSize = 64
net_kwargs={
	"model_name": "varyunet",
	"Block": "ResBottleNeck",
	"planes": 6,
	"in_channels": 1,
	"classes": 1,
	"GridSize": GridSize,
	"layer_nums": [2, 2, 2, 2],
	"adaptor_nums": [2, 2, 2, 2],
	"factor": 2,
	"norm_method": "layer",
	"pool_method": "max",
	"padding": "same",
	"padding_mode": "zeros",
	"end_padding": "valid",
	"end_padding_mode": "zeros",
	"act": "tanh"
}

name = net_kwargs.pop("model_name")
net = CNN(name, net_kwargs)

x = torch.rand(2, 1, GridSize, GridSize)
y = net(x)
print(y.shape)